
# Integracion Entidades SIRIRI-JUSTAT_E
=======================================

En este documento concentramos las acciones preparatorias que deben realizarse para la integración del sistema JUSTAT con el nuevo desarrollo SIRIRI a fin de dar continuidad al sistema estadístico del Superior Tribunal de Justicia. 

In [1]:
#%% Libraries
# Data manipulation and Viz
import pandas as pd
import numpy as np
import os
import nltk
# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib import collections
# machine learning
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as skl_lm
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from neuralprophet import NeuralProphet
from scipy import stats
# others
import bs4
import io
import requests
import json
# initialize some package settings
sns.set(style="whitegrid", color_codes=True, font_scale=1.3)
%matplotlib inline
# Mongo tools
from pymongo import MongoClient
import pymongo
from pprint import pprint
from sklearn import pipeline
import datetime   # This will be needed later
import os
# External ref
# Aggregation Framewok
# https://www.practical-mongodb-aggregations.com/examples/trend-analysis/incremental-analytics.html
# From Mongo to Pandas
# https://docs.mongodb.com/manual/reference/operator/aggregation/setWindowFields/?_ga=2.230420636.1035413487.1644869335-438496097.1644675676
# https://www.mongodb.com/developer/quickstart/pymongoarrow-and-data-analysis/


#%% client = MongoClient()
client = MongoClient('mongodb://10.101.2.97:27017/?readPreference=primary&ssl=false')
bd = client["expedientes_production"]
collections = bd.list_collection_names()

# Mapeo Entidades

Punto de partida de entidades Justat. Nuestro registro un mapeo parcial de entidades, debe consolidarse para tener organoXorgano (etadística e informática). En cualquier caso el listado de entidades u órganos debe ser consistente con las definiciones institucionales de estructura orgánica.

In [14]:
os.getcwd()

'd:\\Users\\sebastian.castillo\\siriri_justat\\scripts'

In [2]:
organismos = bd['organismos'] 

In [9]:
total_organismo = organismos.count_documents({})
print('Total de organos SIRIRI', total_organismo)

Total de organos SIRIRI 212


In [3]:
entidades_justat = pd.read_csv( "d:\\Users\\sebastian.castillo\\siriri_justat\\data\\entidades_justat.csv")

In [17]:
entidades_justat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   organismo              384 non-null    object 
 1   organismo_descripcion  384 non-null    object 
 2   email_oficial          284 non-null    object 
 3   fuero                  384 non-null    object 
 4   circunscripcion        384 non-null    object 
 5   localidad              384 non-null    object 
 6   categoria              75 non-null     float64
 7   tipo                   341 non-null    object 
 8   materia                313 non-null    object 
 9   codigo_organismo       149 non-null    object 
dtypes: float64(1), object(9)
memory usage: 30.1+ KB


Ejemplo Paraná

In [12]:
entidades_justat[entidades_justat.circunscripcion.str.contains('Paraná')]

,organismo,organismo_descripcion,email_oficial,fuero,circunscripcion,localidad,categoria,tipo,materia,codigo_organismo
4,camcco0201pna,Cam Apelaciones II Sala I,camcycs1-pna@jusentrerios.gov.ar,Civil y Comercial,Paraná,Paraná,NaN,cam,cco,parccs01
5,camcco0202pna,Cam Apelaciones II Sala II,camcycs2-pna@jusentrerios.gov.ar,Civil y Comercial,Paraná,Paraná,NaN,cam,cco,parccs02
6,camcco0203pna,Cam Apelaciones II Sala III,camcycs3-pna@jusentrerios.gov.ar,Civil y Comercial,Paraná,Paraná,NaN,cam,cco,parcts02
10,camlab0301pna,Cam Apelaciones III Sala I,camlabs1-pna@jusentrerios.gov.ar,Laboral,Paraná,Paraná,NaN,cam,lab,parcts01
11,camlab0302pna,Cam Apelaciones III Sala II,camlabsala2-pna@jusentrerios.gov.ar,Laboral,Paraná,Paraná,NaN,cam,lab,parctrs02
...,...,...,...,...,...,...,...,...,...,...
376,stjmip0000pna,Mesa Información Permanente,mesainfper@jusentrerios.gov.ar,STJ,Paraná,Paraná,NaN,stj,NaN,NaN
377,stjmui0000pna,Mesa Única Informatizada,mui@jusentrerios.gov.ar,STJ,Paraná,Paraná,NaN,stj,NaN,NaN
378,stjoga0000pna,Oficina de Gestión de Audiencias (OGA),ogaparana@jusentrerios.gov.ar,STJ,Paraná,Paraná,NaN,stj,NaN,NaN
379,stjoma0000pna,Oficina de Medios Alternativos (OMA),omaparana@jusentrerios.gov.ar,STJ,Paraná,Paraná,NaN,stj,NaN,NaN
